In [2]:
cd D:\LAB\ADR\AgenticAdr

D:\LAB\ADR\AgenticAdr


In [ ]:
import os
import subprocess
import tempfile
import shutil
from pathlib import Path
from dotenv import load_dotenv
import re
import json
from typing import List, Dict, Union
from datetime import datetime
from pydantic import BaseModel
import google.generativeai as genai

# --- Configuration ---
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE")

genai.configure(api_key=GOOGLE_API_KEY)

### RepoSummarizer

In [28]:
class RepoSummarizer:
    """
    An agent to summarize a code repository using an LLM.
    """

    def __init__(self, model: genai.GenerativeModel):
        """
        Initializes the summarizer with a specific Gemini model.
        """
        print(f"Initializing with model: {model}")
        self.model = model
        self.ignore_patterns = [
            '.git', '__pycache__', 'node_modules', 'dist', 'build',
            '.DS_Store', '*.pyc', '*.log', '*.tmp', '*.swp'
        ]
        self.text_file_extensions = [
            '.py', '.js', '.ts', '.java', '.c', '.cpp', '.h', '.hpp', '.cs',
            '.go', '.rs', '.rb', '.php', '.html', '.css', '.scss', '.json',
            '.xml', '.yaml', '.yml', '.md', '.txt', '.sh', '.toml', '.ini',
            'Dockerfile', 'Makefile'
        ]


    def _is_text_file(self, file_path):
        """Checks if a file is likely a text file based on its extension."""
        return any(file_path.name.endswith(ext) for ext in self.text_file_extensions)

    def _clone_repo(self, repo_url, temp_dir="../temp"):
        """
        Clones a repository into a temporary directory.
        Returns the path to the temporary directory.
        """
        try:
            # temp_dir = tempfile.mkdtemp()
            print(f"Cloning {repo_url} into {temp_dir}...")
            subprocess.check_call(['git', 'clone', repo_url, temp_dir], stderr=subprocess.DEVNULL)
            print("Cloning successful.")
            return temp_dir
        except subprocess.CalledProcessError as e:
            print(f"Error cloning repository: {e}")
            shutil.rmtree(temp_dir)
            return None
        except FileNotFoundError:
            print("Error: Git is not installed or not in your PATH.")
            shutil.rmtree(temp_dir)
            return None


    def _get_repo_structure(self, repo_path):
        """
        Creates a string representation of the repository's file structure.
        """
        tree_str = ""
        for root, dirs, files in os.walk(repo_path):
            # Filter out ignored directories
            dirs[:] = [d for d in dirs if d not in self.ignore_patterns]
            
            level = root.replace(repo_path, '').count(os.sep)
            indent = ' ' * 4 * level
            tree_str += f"{indent}📁 {os.path.basename(root)}/\n"
            
            sub_indent = ' ' * 4 * (level + 1)
            for f in files:
                if f not in self.ignore_patterns:
                    tree_str += f"{sub_indent}📄 {f}\n"
        return tree_str


    def _get_dependencies(self, repo_path):
        """
        Finds and reads common dependency files.
        """
        dependency_files = {
            "Python": "requirements.txt",
            "Node.js": "package.json",
            "Java (Maven)": "pom.xml",
            "Java (Gradle)": "build.gradle",
            "Ruby": "Gemfile",
        }
        found_deps = "No common dependency files found."
        for tech, filename in dependency_files.items():
            path = Path(repo_path) / filename
            if path.exists():
                try:
                    found_deps = f"--- {tech} Dependencies ({filename}) ---\n"
                    found_deps += path.read_text(encoding='utf-8') + "\n\n"
                except Exception as e:
                    found_deps += f"Could not read {filename}: {e}\n\n"
        return found_deps


    def _summarize_key_files(self, repo_path, num_files=5):
        """
        Finds the largest text files and generates a summary for each.
        """
        file_sizes = []
        for root, _, files in os.walk(repo_path):
            if any(part in root for part in self.ignore_patterns):
                continue
            for file in files:
                file_path = Path(root) / file
                if self._is_text_file(file_path):
                    try:
                        size = file_path.stat().st_size
                        if size > 100: # Ignore very small files
                            file_sizes.append((file_path, size))
                    except FileNotFoundError:
                        continue

        # Sort files by size in descending order and get the top N
        file_sizes.sort(key=lambda x: x[1], reverse=True)
        key_files = [f[0] for f in file_sizes[:num_files]]

        print(f"\nSummarizing the {len(key_files)} largest files...")
        summaries = []
        for file_path in key_files:
            relative_path = file_path.relative_to(repo_path)
            print(f"  - Reading: {relative_path}")
            try:
                content = file_path.read_text(encoding='utf-8')
                if len(content.strip()) == 0:
                    continue

                prompt = f"""
                Analyze the following code from the file '{relative_path}'.
                Provide a concise, high-level summary (2-3 sentences) of its purpose and key functionality.

                ```
                {content[:4000]}
                ```
                """
                response = self.model.generate_content(prompt)
                summaries.append(f"📄 **File: {relative_path}**\n{response.text}\n")
            except Exception as e:
                summaries.append(f"📄 **File: {relative_path}**\n   - Could not summarize: {e}\n")

        return "\n".join(summaries)


    def summarize_repo(self, repo_url):
        """
        The main method to orchestrate the summarization process.
        """
        repo_path = self._clone_repo(repo_url)
        if not repo_path:
            return "Failed to clone the repository. Please check the URL and your Git installation."

        try:
            print("\nAnalyzing repository structure...")
            structure = self._get_repo_structure(repo_path)

            print("\nLooking for dependency files...")
            dependencies = self._get_dependencies(repo_path)

            file_summaries = self._summarize_key_files(repo_path)

            print("\nGenerating final high-level summary...")
            final_prompt = f"""
            You are a senior software architect tasked with analyzing a new code repository.
            Based on the information provided below, generate a comprehensive, high-level summary.

            Your summary should include:
            1.  **Project Purpose**: What is the likely goal of this project?
            2.  **Technology Stack**: What languages, frameworks, and key libraries are used?
            3.  **Architecture Overview**: Briefly describe the project's structure and how the key files interact.

            --- REPOSITORY INFORMATION ---

            **1. File & Directory Structure:**
            ```
            {structure[:4000]}
            ```

            **2. Detected Dependencies:**
            ```
            {dependencies}
            ```

            **3. Summaries of Key Files:**
            ```
            {file_summaries}
            ```
            --- END OF INFORMATION ---

            Please provide the final summary in a clear, well-structured markdown format.
            """

            final_response = self.model.generate_content(final_prompt)
            return final_response.text

        finally:
            pass
            # Clean up the temporary directory
            # print(f"\nCleaning up temporary directory: {repo_path}")
            # shutil.rmtree(repo_path)



In [30]:


# A simple, well-known Python repository
# repo_url_to_summarize = "https://github.com/requests/requests.git"

# A more complex JavaScript repository
# repo_url_to_summarize = "https://github.com/axios/axios.git"

# A Go repository
# repo_url_to_summarize = "https://github.com/gin-gonic/gin.git"

llm_model = genai.GenerativeModel('gemini-2.5-flash')


# Sismics music
repo_url_to_summarize = "https://github.com/sismics/music.git"
summarizer_agent = RepoSummarizer(model=llm_model)
summary = summarizer_agent.summarize_repo(repo_url_to_summarize)

print("\n" + "="*50)
print("🚀 FINAL REPOSITORY SUMMARY")
print("="*50)
print(summary)

Initializing with model: genai.GenerativeModel(
    model_name='models/gemini-2.5-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)
Cloning https://github.com/sismics/music.git into ../temp...
Cloning successful.

Analyzing repository structure...

Looking for dependency files...

Summarizing the 5 largest files...
  - Reading: music-web\src\main\webapp\src\lib\angular.js
  - Reading: music-web\src\main\webapp\src\lib\less.js
  - Reading: music-web\src\main\webapp\src\lib\jquery.js
  - Reading: music-web\src\main\webapp\src\lib\angular.ui-bootstrap.js
  - Reading: music-web\src\main\webapp\src\style\bootstrap.css

Generating final high-level summary...

🚀 FINAL REPOSITORY SUMMARY
Based on the provided information, here's a comprehensive, high-level summary of the code repository:

## Project Summary: Music Management and Playback System

This repository appears to house a multi-platform, comprehensive music 

### AdrWriterAgent

In [25]:
# --- Data Model for an ADR ---
class ADR(BaseModel):
    """Represents a single Architecture Decision Record."""
    title: str
    context: str
    decision: str
    # Accept either a simple string or a structured dictionary for consequences
    consequences: Union[str, Dict[str, List[str]]]

# --- ADR Writer Agent ---
class AdrWriterAgent:
    """
    An agent that analyzes a repository summary to identify, format,
    and save key design decisions as Architecture Decision Records (ADRs).
    """
    def __init__(self, model: genai.GenerativeModel):
        """
        Initializes the AdrWriterAgent.

        Args:
            model: An initialized generative model instance.
        """
        print("Initializing AdrWriterAgent...")
        self.model = model

    def _sanitize_filename(self, name: str) -> str:
        """
        Sanitizes a string to be used as a valid filename (internal helper method).
        """
        name = name.replace(' ', '_')
        name = re.sub(r'[^\w\.-]', '', name)
        return name[:100] # Truncate to a reasonable length

    def _extract_design_decisions(self, summary: str) -> List[Dict]:
        """
        Uses the LLM to extract important design decisions from a repository summary.
        """
        # (This method remains the same as before)
        print("Extracting design decisions from the summary...")
        prompt = f"""
        You are an expert senior software architect. Your task is to analyze a repository summary
        and identify the most critical architectural and technological design decisions made in the project.

        From the summary provided below, extract at least three major design decisions.
        For each decision, you must provide:
        - A short, clear 'title' for the decision.
        - The 'context' explaining the problem or requirement that necessitated this decision.
        - The specific 'decision' that was made.
        - The 'consequences' of this decision. You can provide this as a simple string or as a JSON object with "pros" and "cons" keys.

        **Repository Summary**:
        ---
        {summary}
        ---

        Return your response as a valid JSON array of objects. Do not include any text outside of the JSON array.
        """
        response = self.model.generate_content(prompt)
        clean_response = response.text.strip().replace("```json", "").replace("```", "").strip()
        try:
            decisions = json.loads(clean_response)
            print(f"Successfully extracted {len(decisions)} design decisions.")
            return decisions
        except json.JSONDecodeError:
            print(f"Error: The model did not return a valid JSON. Raw response:\n{clean_response}")
            raise ValueError("LLM response could not be decoded as JSON.")

    def _format_and_save_adrs(self, adrs: List[ADR], output_path: str) -> List[str]:
        """
        Formats a list of ADRs, saves each to a markdown file, and returns the list of formatted strings.
        """
        if not adrs:
            print("No ADRs to process.")
            return []

        # Ensure the output directory exists
        os.makedirs(output_path, exist_ok=True)
        print(f"Saving ADRs to directory: '{os.path.abspath(output_path)}'")

        formatted_adrs_list = []
        for i, adr in enumerate(adrs, start=1):
            consequences_text = ""
            if isinstance(adr.consequences, dict):
                pros = adr.consequences.get("pros", [])
                cons = adr.consequences.get("cons", [])
                if pros:
                    consequences_text += "**Pros:**\n" + "\n".join(f"- {item}" for item in pros) + "\n\n"
                if cons:
                    consequences_text += "**Cons:**\n" + "\n".join(f"- {item}" for item in cons)
            else:
                consequences_text = adr.consequences

            adr_text = f"""# ADR-{i:03d}: {adr.title}

**Date**: {datetime.now().strftime('%Y-%m-%d')}
**Status**: Proposed

## Context
{adr.context}

## Decision
{adr.decision}

## Consequences
{consequences_text.strip()}
"""
            formatted_adrs_list.append(adr_text.strip())

            # File Saving Logic
            safe_filename = self._sanitize_filename(adr.title)
            filename = f"{i:03d}_{safe_filename}.md"
            file_path = os.path.join(output_path, filename)
            try:
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(adr_text)
                print(f"  -> Successfully saved {filename}")
            except IOError as e:
                print(f"  -> Failed to save {filename}. Error: {e}")

        print("Formatting and saving complete.")
        return formatted_adrs_list

    def write_and_save_adrs(self, summary: str, output_path: str) -> List[str]:
        """
        The main public method to orchestrate the entire ADR generation process.

        Args:
            summary: The repository summary text.
            output_path: The directory where the markdown ADR files will be saved.

        Returns:
            A list of strings, where each string is the content of one formatted ADR.
        """
        if not summary or not summary.strip():
            print("Warning: The provided summary is empty. No ADRs can be generated.")
            return []
        
        # 1. Extract structured data from the summary
        decisions_data = self._extract_design_decisions(summary)
        
        # 2. Convert the data into validated Pydantic models
        adrs = [ADR(**data) for data in decisions_data]
        
        # 3. Format, save, and return the ADRs
        return self._format_and_save_adrs(adrs, output_path)



In [ ]:
# 2. Initialize the model and the agent
model = genai.GenerativeModel('gemini-2.5-flash')
adr_agent = AdrWriterAgent(model=model)

adr_output_directory = "Generated_ADRs"

# 4. Run the agent's main method
list_of_adr_strings = adr_agent.write_and_save_adrs(
    summary=summary,
    output_path=adr_output_directory
)

# 5. The results are now in the returned list and saved to disk
if list_of_adr_strings:
    print(f"\n✅ Process complete. Returned a list with {len(list_of_adr_strings)} ADRs.")
    print(f"You can find the markdown files in the '{adr_output_directory}' folder.")

Initializing AdrWriterAgent...
Extracting design decisions from the summary...
Successfully extracted 3 design decisions.
Saving ADRs to directory: 'D:\LAB\ADR\AgenticAdr\Generated_ADRs'
  -> Successfully saved 001_Self-Hosted_Multi-Platform_Architecture.md
  -> Successfully saved 002_Java_as_Primary_Backend__Desktop_Language.md
  -> Successfully saved 003_Embedded_H2_Database_for_Data_Persistence.md
Formatting and saving complete.

✅ Process complete. Returned a list with 3 ADRs.
You can find the markdown files in the 'Generated_ADRs' folder.


### Orchestrator

In [26]:
class OrchestratorAgent:
    """
    Orchestrates the workflow of summarizing a code repository
    and generating Architecture Decision Records (ADRs) from the summary.
    """
    def __init__(self, model: genai.GenerativeModel):
        """
        Initializes the orchestrator and its subordinate agents.

        Args:
            model: An initialized generative model instance to be used by the AdrWriterAgent.
        """
        print("Initializing the Orchestrator Agent...")
        # The orchestrator creates and manages its own instances of the other agents
        self.summarizer_agent = RepoSummarizer(model=model)
        self.adr_writer_agent = AdrWriterAgent(model=model)
        print("Orchestrator is ready.")

    def run(self, repo_url: str, adr_output_path: str):
        """
        Executes the full end-to-end workflow.

        1. Clones and summarizes the repository.
        2. Generates and saves ADRs based on the summary.

        Args:
            repo_url: The URL of the Git repository to analyze.
            adr_output_path: The directory path to save the generated ADR files.

        Returns:
            A list of strings, where each string is the content of a generated ADR,
            or an empty list if the process fails.
        """
        print("\n" + "="*50)
        print("🚀 Starting Orchestration Workflow")
        print("="*50)

        # --- STEP 1: Summarize the Repository ---
        print("\n[STEP 1/2] Handing off to RepoSummarizer Agent...")
        summary = self.summarizer_agent.summarize_repo(repo_url)

        if not summary or not summary.strip():
            print("\nWorkflow halted: RepoSummarizer did not return a valid summary.")
            return []

        print("\n✅ RepoSummarizer completed its task.")

        # --- STEP 2: Generate and Save ADRs ---
        print("\n[STEP 2/2] Handing off to AdrWriter Agent...")
        list_of_adr_strings = self.adr_writer_agent.write_and_save_adrs(
            summary=summary,
            output_path=adr_output_path
        )

        print("\n✅ AdrWriter completed its task.")
        print("\n" + "="*50)
        print("🎉 Orchestration Workflow Finished Successfully!")
        print("="*50)

        return list_of_adr_strings



In [27]:
# --- EXAMPLE USAGE OF THE ORCHESTRATOR ---
if __name__ == '__main__':

    # 1. Define your inputs
    repo_url_to_process = "https://github.com/sismics/music.git"
    output_directory_for_adrs = "Generated_ADRs"

    # 2. Initialize the generative model
    # It's recommended to use a powerful model for these reasoning tasks
    llm_model = genai.GenerativeModel('gemini-2.5-flash')

    # 3. Instantiate the Orchestrator Agent
    orchestrator = OrchestratorAgent(model=llm_model)

    # 4. Execute the entire workflow with a single call
    final_adrs = orchestrator.run(
        repo_url=repo_url_to_process,
        adr_output_path=output_directory_for_adrs
    )

    # 5. Final confirmation
    if final_adrs:
        print(f"\nProcess complete. {len(final_adrs)} ADRs were generated and saved.")
        print(f"You can find the markdown files in the '{output_directory_for_adrs}' folder.")
    else:
        print("\nProcess finished, but no ADRs were generated.")

Initializing the Orchestrator Agent...
Initializing with model: genai.GenerativeModel(
    model_name='models/gemini-2.5-flash',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)
Initializing AdrWriterAgent...
Orchestrator is ready.

🚀 Starting Orchestration Workflow

[STEP 1/2] Handing off to RepoSummarizer Agent...
Cloning https://github.com/sismics/music.git into C:\Users\rudra\AppData\Local\Temp\tmp_bql5qpp...
Cloning successful.

Analyzing repository structure...

Looking for dependency files...

Summarizing the 5 largest files...
  - Reading: music-web\src\main\webapp\src\lib\angular.js
  - Reading: music-web\src\main\webapp\src\lib\less.js
  - Reading: music-web\src\main\webapp\src\lib\jquery.js
  - Reading: music-web\src\main\webapp\src\lib\angular.ui-bootstrap.js
  - Reading: music-web\src\main\webapp\src\style\bootstrap.css

Generating final high-level summary...

✅ RepoSummarizer completed its task.

[STE